<img src="images/bannerugentdwengo.png" alt="Banner" width="400"/>

<div>
    <font color=#690027 markdown="1">
        <h1>DATA FRAME</h1>    
    </font>
</div>

<div class="alert alert-box alert-success">
In deze notebook leer je hoe je de data voorverwerkt. Je zal o.a. leren hoe je categorische variabelen omzet naar numerieke en hoe je overtollige of onvolledige gegevens verwijdert. 
</div>

Je beschikt over een dataset, die je vindt in het betand `data.csv` in de map `data`. 

Je kan het csv-bestand bekijken als je het bv. opent met Kladblok (zie Figuur 1).

<img src="images/datacsv.png" alt="Banner" width="900"/>
<center>Figuur 1: data.csv geopend met Kladblok.</center>

Dit bestand bevat (fictieve) patiëntengegevens: leeftijd, geslacht, woonplaats, hartslag, bloedgroep met rhesus, hoe ernstig de klachten van de patiënt zijn, of de patiënt pijn heeft en zich ziek voelt, of het aantal symptomen toeneemt of niet, en het resultaat van de meting van een bepaalde parameter. <br>Elke regel komt overeen met een bepaalde patiënt.

#### Opdracht
- Over hoeveel patiënten gaat het hier?
- Van twee patiënten ontbreken er gegevens. Van welke patiënten?
- Welke gegevens zijn numeriek en welke zijn categorisch (niet-numeriek)? 

Antwoord:

### Nodige modules importeren

In [ ]:
import numpy as np
import pandas as pd

### Gegevens inlezen met pandas en laten zien 

In [ ]:
data = pd.read_csv("data/data.csv") 

De functie `read_csv` van pandas leest het csv-bestand in en maakt een *data frame* aan: een tabel die al de gegevens uit het csv-bestand bevat.<br>Hierbij worden de namen van de gegevens (indien voorhanden) duidelijk weergegeven als titels van de kolommen en worden de rijen genummerd (te beginnen vanaf 0). Een data frame is een **tweedimensionale gegevensstructuur**.

In [ ]:
data

Deze tabel bevat dus de patiëntengegevens, een kolom per gegeven, en elke rij komt overeen met een bepaalde patiënt.


Hier en daar in de tabel zie je `NaN` (*not a number*) staan, wat betekent dat op die plaats in de tabel een waarde ontbreekt. Dit komt overeen met de lege plaatsen in het csv-bestand (de patiënt van regel 10 staat op de rij met nummer 8 in het pandas data frame en de patiënt van regel 13 staat op de rij met nummer 11 in het pandas data frame).

### Gegevens voorverwerken 

- Computers rekenen. Daarvoor zijn dus getallen nodig. Daarom moet je waarden van de categorische (de niet-numerieke) variabelen omzetten naar numerieke waarden, voordat je die gegevens door de computer kunt laten verwerken. 
- De woonplaats van een patiënt is niet relevant voor een behandeling, dus die kolom zal je ook verwijderen.
- De rijen waarop `NaN` staat, zal je verwijderen, opdat de computer niet zou vastlopen omdat er waarden ontbreken.

In [ ]:
# categorische variabelen numeriek maken a.d.h.v. methode replace()
# keuze cijfers is vrij, best zinvolle keuze maken
data["Geslacht"] = data["Geslacht"].replace("V", 0)
data["Geslacht"] = data["Geslacht"].replace("M", 1)
data["Bloedgroep"] = data["Bloedgroep"].replace("O", 0)
data["Bloedgroep"] = data["Bloedgroep"].replace("A", 1)
data["Bloedgroep"] = data["Bloedgroep"].replace("B", 2)
data["Bloedgroep"] = data["Bloedgroep"].replace("AB", 3)
data["Rhesus"] = data["Rhesus"].replace("+", 1)
data["Rhesus"] = data["Rhesus"].replace("-", -1)
data["TypeKlacht"] = data["TypeKlacht"].replace("Standaard", 1)
data["TypeKlacht"] = data["TypeKlacht"].replace("NietErnstig", 0)
data["TypeKlacht"] = data["TypeKlacht"].replace("Ernstig", 2)
data["TypeKlacht"] = data["TypeKlacht"].replace("Verontrustend", 3)
data["Pijn"] = data["Pijn"].replace("N", 0)
data["Pijn"] = data["Pijn"].replace("J", 1)
data["AantalSymptomen"] = data["AantalSymptomen"].replace("Constant", 0)
data["AantalSymptomen"] = data["AantalSymptomen"].replace("Dalend", -1)
data["AantalSymptomen"] = data["AantalSymptomen"].replace("Stijgend", 1)

In [ ]:
data

In [ ]:
# overbodige kolom verwijderen a.d.h.v. sleutelwoord del
del data["Gemeente"]

In [ ]:
data

In [ ]:
# rijen met onvolledige gegevens verwijderen met methode dropna()
data = data.dropna()

In [ ]:
data

### Omzetten naar NumPy array

Je werkt vaak met NumPy om de gegevens te laten verwerken, bv. om er informatie uit te halen. Na de voorverwerking is het data frame klaar om het om te zetten naar een NumPy array. 

In [ ]:
data = np.array(data)

In [ ]:
data

In [ ]:
# parameters en klasse onderscheiden
parameters = data[:, :9]        # eerste 9 kolommen van matrix zijn beschouwde parameters
klasse = data[:, 9]             # laatste kolom is klasse waartoe persoon behoort

In [ ]:
print(parameters)

In [ ]:
print(klasse)

### Referentie

Wes McKinney. (2018). *Python for Data Analysis.* O'Reilly.

<img src="images/cclic.png" alt="Banner" align="left" width="100"/><br><br>
Notebook AI in de Zorg, zie <a href="http://www.aiopschool.be">AI Op School</a>, van F. wyffels & N. Gesquière, is in licentie gegeven volgens een <a href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Naamsvermelding-NietCommercieel-GelijkDelen 4.0 Internationaal-licentie</a>. 